# 0 data & packages

## 0-0 환경 확인

In [1]:
# 컴퓨터 환경
import platform
print("운영 체제:", platform.system())
print("OS 버전:", platform.version())
print("프로세서:", platform.processor())
print("세부 정보:", platform.platform())

운영 체제: Windows
OS 버전: 10.0.26100
프로세서: Intel64 Family 6 Model 154 Stepping 3, GenuineIntel
세부 정보: Windows-11-10.0.26100-SP0


In [3]:
# CPU 코어 수 확인
import multiprocessing
cpu_cores = multiprocessing.cpu_count()
print(f"이 시스템의 CPU 코어 수: {cpu_cores}")

이 시스템의 CPU 코어 수: 20


In [4]:
# Jupyter Notebook 버전 확인
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.25.0
ipykernel        : 6.28.0
ipywidgets       : 7.8.1
jupyter_client   : 8.6.0
jupyter_core     : 5.7.2
jupyter_server   : 2.14.1
jupyterlab       : 4.0.11
nbclient         : 0.8.0
nbconvert        : 7.10.0
nbformat         : 5.9.2
notebook         : 7.0.8
qtconsole        : 5.5.1
traitlets        : 5.14.3


In [6]:
# 파이썬 버전
import sys
print("Python 버전:", sys.version)

Python 버전: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]


## 0-1 패키지 설치 및 불러오기

In [ ]:
! pip install holidays

In [ ]:
! pip install pyproj

In [ ]:
! pip install numba

In [ ]:
! pip install geopy

In [ ]:
! pip install joblib

In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time

# 경고 무시
warnings.filterwarnings("ignore")


from pyproj import Proj, Transformer
import holidays
from joblib import Parallel, delayed
from geopy.distance import geodesic  # 이 부분은 Numba가 아닌 기본 방식으로 처리
from numba import njit  # Numba 사용

In [ ]:
# 데이터를 불러올 경로 지정
'''아래 따옴표 안에 데이터가 저장되어있는 경로 작성'''
path = ''

## 0-2 데이터 불러오기

### (1) 제공된 데이터 - main
데이터명은 아래와 같이 간소화하여 사용

- 유동인구 → 유동인구
- 카드매출 (cell id별) → 카드매출_가맹점
- 카드매출 (지역별 업종 유입고객) → 카드매출_유입고객**

In [ ]:
# 유동인구
list_ = [2307, 2308, 2309, 2310, 2311, 2312, 2401, 2402, 2403, 2404, 2405, 2406]
for i in list_:
    globals()[f'popul_{i}'] = pd.read_csv(os.path.join(path, f'50CELL_DJJUNGGU_20{i}.csv'))

In [360]:
# 카드매출_가맹점
list_ = [2307, 2308, 2309, 2310, 2311, 2312, 2401, 2402, 2403, 2404, 2405, 2406]
for i in list_:
    globals()[f'card_biz_{i}'] = pd.read_csv(os.path.join(path, f'카드매출_가맹점/대전중구_set1_20{i}.csv'), encoding='cp949')

In [57]:
# 카드매출_유입고객
list_ = [2307, 2308, 2309, 2310, 2311, 2312, 2401, 2402, 2403, 2404, 2405, 2406]
for i in list_:
    globals()[f'card_cus_{i}'] = pd.read_csv(os.path.join(path, f'카드매출_유입고객/대전중구_set2_20{i}.csv'), encoding='cp949')

In [ ]:
# 데이터 합치기
data_popul = pd.concat(eval(f'popul_{i}') for i in list_)
data_biz= pd.concat(eval(f'card_biz_{i}') for i in list_)
data_cus = pd.concat(eval(f'card_cus_{i}') for i in list_)

### (2) 제공된 데이터 - sub (컬럼정의서, 업종분류 등)

In [13]:
data_loc = pd.read_csv(os.path.join(path, '국토지리정보원_국토조사_(격자)500M.csv'))
data_sort = pd.read_excel(os.path.join(path, '상가(상권)정보_업종분류.xlsx'), skiprows=1)

### (3) 외부 데이터
- 행정동 코드 : 행정동 코드에 맞는 행정동명 찾을 때 활용
- 한국표준산업분류10차 : 업종 분류 코드에 맞는 항목명 찾을 때 활용

In [15]:
# 행정동 코드
data_admin = pd.read_excel(os.path.join(path,'KIKcd_H.20240801.xlsx'))

In [16]:
# 한국표준산업분류10차
data_code = pd.read_excel(os.path.join(path, '한국표준산업분류10차_표.xlsx'), skiprows=2)

# 1 데이터 전처리

## 1-1 유동인구

In [15]:
data_popul.head()

,id,x,y,timezn_cd,m00,m10,m15,m20,m25,m30,...,f40,f45,f50,f55,f60,f65,f70,total,admi_cd,etl_ymd
0,58904249,343802,408793,7,0.00,0.01,0.01,0.01,0.04,0.06,...,0.00,0.03,0.05,0.04,0.02,0.03,0.06,1.45,30140740,20230722
1,58904249,343802,408793,7,0.00,0.01,0.25,0.02,0.04,0.02,...,0.01,0.05,0.05,0.05,0.03,0.05,0.01,1.67,30140740,20230728
2,58904249,343802,408793,7,0.00,0.01,0.32,0.02,0.02,0.04,...,0.01,0.04,0.05,0.04,0.02,0.02,0.03,1.63,30140740,20230724
3,58904249,343802,408793,7,0.00,0.01,0.37,0.01,0.03,0.04,...,0.04,0.06,0.06,0.07,0.02,0.02,0.03,1.88,30140740,20230710
4,58904249,343802,408793,8,0.01,0.03,0.02,0.05,0.03,0.01,...,0.06,0.09,0.05,0.04,0.11,0.03,0.03,1.44,30140740,20230730


### (1) id 

In [17]:
# 고유값 확인
data_popul['id'].nunique()

10993

In [18]:
# 결측치 확인
data_popul['id'].isna().sum()

0

#### ① id와 (x, y) 조합이 일대일대응되는지 확인
- *유동인구 정의서* 에서 id는 셀id라고 명시되어있음
- 그렇다면 50m*50m 셀에 각각 번호를 부여한 것인데 **id와 (x, y) 조합이 일대일대응 되는지** 확인

In [24]:
# 확인을 위해 필요한 컬럼만 추출
df_sampled = data_popul[['etl_ymd', 'x', 'y', 'id']]

# 각 행별로 x, y를 x_y라는 컬럼에 할당 = (x,y) 조합 생성
df_sampled['x_y'] = list(zip(data_popul['x'], data_popul['y']))

# x_y와 id 두 컬럼 모두에서 중복인 값 제거
# df_sampled_unique = df_sampled.drop_duplicates(subset=['x_y', 'id'])

# 결과 확인
print(f'id 고유값 개수 : {df_sampled['id'].nunique()}')
print(f'(x, y) 조합의 개수 : {f_sampled['x_y'].nunique()}')

C:\Users\jinhe\AppData\Local\Temp\ipykernel_14708\774880757.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sampled['x_y'] = list(zip(data_popul_admin1['x'], data_popul_admin1['y']))


id 고유값 개수 : 10197
(x, y) 조합의 개수 : 10197


**일대일 대응됨**

### (2) 행정동 코드

In [21]:
# 고유값 확인
data_popul['admi_cd'].unique()

array([30140740, 30140690, 30140700, 30140730, 30140710, 30140680,
       30140655, 30140670, 30140720, 30140550, 30140560, 30140535,
       30140630, 30140575, 30140640, 30140605, 30140620], dtype=int64)

- 모두 대전광역시 중구의 행정동 코드

#### ① 행정동명 컬럼 추가
- 행정안전부 *KIKcd_H.20240801.xlsx* 활용

In [33]:
# 데이터 불러오기
data_admin = pd.read_excel(os.path.join(path, 'KIKcd_H.20240801.xlsx'))

In [35]:
# 데이터 확인
data_admin.head()

,행정동코드,시도명,시군구명,읍면동명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,20081101,NaN
3,1111053000,서울특별시,종로구,사직동,19880423,NaN
4,1111054000,서울특별시,종로구,삼청동,19880423,NaN


In [37]:
# 대전 중구 관련 데이터만 남기기
data_admin = data_admin[['행정동코드', '시도명', '시군구명', '읍면동명']]
data_admin = data_admin[(data_admin['시군구명'] == '중구') & (data_admin['시도명'] == '대전광역시')]
data_admin.drop(['시도명', '시군구명'], axis=1, inplace=True)
data_admin

,행정동코드,읍면동명
1130,3014000000,NaN
1131,3014053500,은행선화동
1132,3014055000,목동
1133,3014056000,중촌동
1134,3014057500,대흥동
1135,3014060500,문창동
1136,3014062000,석교동
1137,3014063000,대사동
1138,3014064000,부사동
1139,3014065500,용두동


In [41]:
# mapping할 딕셔너리 만들기
list_code_num = data_popul['admi_cd'].unique().tolist()
list_name = []

# 행정동 코드 str 타입으로 변환
data_admin['행정동코드'] = data_admin['행정동코드'].astype('str')

for i in data_popul['admi_cd'].unique().astype('str').tolist():
  list_name.append(data_admin[data_admin['행정동코드'].str.contains(i)].iloc[0]['읍면동명'])

# 딕셔너리 생성
admin_cd= dict(zip(list_code_num, list_name))


# 컬럼 붙이기
data_popul_admin = data_popul.copy()
data_popul_admin['행정동명'] = data_popul_admin['admi_cd'].map(admin_cd)

In [45]:
# 데이터 확인
data_popul_admin.head()

,id,x,y,timezn_cd,m00,m10,m15,m20,m25,m30,...,f45,f50,f55,f60,f65,f70,total,admi_cd,etl_ymd,행정동명
0,58904249,343802,408793,7,0.00,0.01,0.01,0.01,0.04,0.06,...,0.03,0.05,0.04,0.02,0.03,0.06,1.45,30140740,20230722,산성동
1,58904249,343802,408793,7,0.00,0.01,0.25,0.02,0.04,0.02,...,0.05,0.05,0.05,0.03,0.05,0.01,1.67,30140740,20230728,산성동
2,58904249,343802,408793,7,0.00,0.01,0.32,0.02,0.02,0.04,...,0.04,0.05,0.04,0.02,0.02,0.03,1.63,30140740,20230724,산성동
3,58904249,343802,408793,7,0.00,0.01,0.37,0.01,0.03,0.04,...,0.06,0.06,0.07,0.02,0.02,0.03,1.88,30140740,20230710,산성동
4,58904249,343802,408793,8,0.01,0.03,0.02,0.05,0.03,0.01,...,0.09,0.05,0.04,0.11,0.03,0.03,1.44,30140740,20230730,산성동


### (3) x, y 좌표
- KT에서 사용한 KATECH 좌표를 경위도로 변환
- 변환식은 대전중구청 담당자분께 받은 정보 참고 (+ 대회 홈페이지>공지사항>Q&A 답변)

In [5]:
# 변환 함수 정의
katech_proj = Proj(
    "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel "
    "+towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43"
)

# Define the WGS84 projection
wgs84_proj = Proj(proj="latlong", datum="WGS84")

# Create a Transformer object for the conversion
transformer = Transformer.from_proj(katech_proj, wgs84_proj)

# Function to convert from KATECH (TM127) to WGS84
def katech_to_wgs84(x, y):
    lon, lat = transformer.transform(x, y)
    return lat, lon


# 경위도로 변환
data_popul_admin['latitude'], data_popul_admin['longitude'] = zip(*data_popul_admin.apply(lambda row: katech_to_wgs84(row['x'], row['y']), axis=1))

### (4) 필요없는 컬럼 삭제
- 'x', 'y' : 경위도로 변환 완료 (latitude, longitude로 대체)
- 'admi_cd' : 행정동명 컬럼 추가 완료 (행정동명으로 대체)
- 'total' : 연령대별 이동경로 및 소비패턴을 살펴볼 예정이므로 삭제

In [11]:
data_popul_admin.drop(['x', 'y', 'admi_cd', 'total'], axis=1, inplace=True)
data_popul_admin.head()

,id,timezn_cd,m00,m10,m15,m20,m25,m30,m35,m40,...,f45,f50,f55,f60,f65,f70,etl_ymd,행정동명,latitude,longitude
0,59026163,0,0.0,0.0,0.32,0.02,0.02,0.02,0.02,0.11,...,0.0,0.07,0.09,0.05,0.02,0.02,2023-07-01,산성동,36.288905,127.377754
1,59050541,0,0.0,0.0,0.22,0.02,0.02,0.02,0.02,0.08,...,0.0,0.05,0.06,0.03,0.02,0.02,2023-07-01,산성동,36.288911,127.378868
2,59050544,0,0.0,0.0,0.34,0.02,0.02,0.02,0.02,0.12,...,0.0,0.07,0.10,0.05,0.02,0.02,2023-07-01,산성동,36.290263,127.378857
3,59050545,0,0.0,0.0,0.37,0.03,0.03,0.03,0.03,0.13,...,0.0,0.08,0.11,0.05,0.03,0.03,2023-07-01,산성동,36.290714,127.378853
4,59074922,0,0.0,0.0,0.36,0.03,0.03,0.03,0.03,0.13,...,0.0,0.08,0.10,0.05,0.03,0.03,2023-07-01,산성동,36.290269,127.379970


### (5) "공휴일" 컬럼 추가
- 국가에서 지정한 공휴일
- holidays 패키지 활용
- 공휴일 아님 / 공휴일

In [13]:
# etl_ymd을 datetime으로
data_popul_admin['etl_ymd'] = pd.to_datetime(data_popul_admin['etl_ymd'])

In [14]:
## 공휴일
import holidays

# 한국 휴일 객체 생성
kr_holidays = holidays.KR()

# holiday 컬럼 추가
data_popul_admin['공휴일'] = data_popul_admin['etl_ymd'].apply(lambda x: '공휴일' if x in kr_holidays else '공휴일 아님')

### (6) "주말" 컬럼 추가
- *etl_ymd* 에서 파생
- 월요일 ~ 금요일 : 평일 / 토요일, 일요일 : 주말

In [15]:
## 주말
def weekend(data):
    result = data.dayofweek
    if result >= 5:     # 5 : 토 / 6 : 일
        return '주말'
    else:
        return '평일'

# 공휴일이 아닌 데이터에만 채워넣기 (공휴일 건들면 안되니까)
data_popul_admin['주말'] = data_popul_admin['etl_ymd'].apply(weekend)

### (7) 연령대별 컬럼 합치기 : 10세 단위로
- m00, f00 : 10세 미만
- m10 ~ m15, f10 ~ f15 : 10대
- m20 ~ m25, f20 ~ f25 : 20대
- m30 ~ m35, f30 ~ f35 : 30대
- m40 ~ m45, f40 ~ f45 : 40대
- m50 ~ m55, f50 ~ f55 : 50대
- m60 ~ m65, f60 ~ f65 : 60대
- m70, f70 : 70대 이상

In [19]:
# 10세 단위로 묶기 (m00, f00은 0~9세이므로 해당 작업에 포함x)
data_popul_admin['m10'] = data_popul_admin['m10'] + data_popul_admin['m15']
data_popul_admin['m20'] = data_popul_admin['m20'] + data_popul_admin['m25']
data_popul_admin['m30'] = data_popul_admin['m30'] + data_popul_admin['m35']
data_popul_admin['m40'] = data_popul_admin['m40'] + data_popul_admin['m45']
data_popul_admin['m50'] = data_popul_admin['m50'] + data_popul_admin['m55']
data_popul_admin['m60'] = data_popul_admin['m60'] + data_popul_admin['m65']

data_popul_admin['f10'] = data_popul_admin['f10'] + data_popul_admin['f15']
data_popul_admin['f20'] = data_popul_admin['f20'] + data_popul_admin['f25']
data_popul_admin['f30'] = data_popul_admin['f30'] + data_popul_admin['f35']
data_popul_admin['f40'] = data_popul_admin['f40'] + data_popul_admin['f45']
data_popul_admin['f50'] = data_popul_admin['f50'] + data_popul_admin['f55']
data_popul_admin['f60'] = data_popul_admin['f60'] + data_popul_admin['f65']

In [20]:
# 필요한 연령대만 남기기 & 컬럼 순서조정
data_popul_admin = data_popul_admin[['id', 'etl_ymd', 'timezn_cd','latitude', 'longitude','행정동명',
                                         'm00', 'm10', 'm20', 'm30', 'm40', 'm50', 'm60', 'm70', 
                                         'f00', 'f10','f20', 'f30', 'f40', 'f50', 'f60', 'f70',
                                         '공휴일','주말']]

In [22]:
data_popul_admin.head()

,id,etl_ymd,timezn_cd,latitude,longitude,행정동명,m00,m10,m20,m30,...,f00,f10,f20,f30,f40,f50,f60,f70,공휴일,주말
0,59026163,2023-07-01,0,36.288905,127.377754,산성동,0.0,0.32,0.04,0.04,...,0.0,0.0,0.12,0.02,0.16,0.16,0.07,0.02,공휴일 아님,주말
1,59050541,2023-07-01,0,36.288911,127.378868,산성동,0.0,0.22,0.04,0.04,...,0.0,0.0,0.08,0.02,0.11,0.11,0.05,0.02,공휴일 아님,주말
2,59050544,2023-07-01,0,36.290263,127.378857,산성동,0.0,0.34,0.04,0.04,...,0.0,0.0,0.12,0.02,0.17,0.17,0.07,0.02,공휴일 아님,주말
3,59050545,2023-07-01,0,36.290714,127.378853,산성동,0.0,0.37,0.06,0.06,...,0.0,0.0,0.13,0.03,0.19,0.19,0.08,0.03,공휴일 아님,주말
4,59074922,2023-07-01,0,36.290269,127.379970,산성동,0.0,0.36,0.06,0.06,...,0.0,0.0,0.13,0.03,0.18,0.18,0.08,0.03,공휴일 아님,주말


### (8) 전처리 완료 데이터 저장
- 2. 이동경로 분석 부터는 전처리 완료된 데이터를 불러와서 사용

In [ ]:
data_popul_admin.to_csv(os.path.join(path, '유동인구_전처리_완료.csv'), encoding = 'cp949', index=False)

## 1-2 카드매출_가맹점

In [364]:
# *카드매출_가맹점*
data_biz.isna().sum()

기준일자               0
cell_id            0
업종대분류          15652
업종중분류          23604
업종소분류         178261
이용건수               0
이용건수_지역화폐          0
이용건수_재난지원금         0
이용건수_기타            0
이용금액               0
이용금액_지역화폐          0
이용금액_재난지원금         0
이용금액_기타            0
dtype: int64

### (1) cell_id : 경위도 컬럼 추가
- 500m*500m 셀의 위치를 나타내는 컬럼
- 함께 주어진 ***국토지리정보원_국토조사_(격자)500M.csv***를 활용하여 경도/위도로 변환

#### ① *국토지리정보원_국토조사_(격자)500M.csv*
> x좌표, y좌표 : UTM-K 좌표 → 경위도 좌표(wsg84)로 변환  
> (*이때, EPSG는 중앙 경선이 127도가 되도록 하는 **대한민국 중앙부 기준** 사용)

In [19]:
# 국토지리정보원_국토조사_(격자)500M
# x, y → 경위도

transformer = Transformer.from_crs("epsg:5178", "epsg:4326", always_xy=True)

# 위도와 경도로 변환하는 함수
def katec_to_wgs84(x, y):
    lon, lat = transformer.transform(x, y)
    return lon, lat

# 새로운 열을 추가
data_loc['longitude'], data_loc['latitude'] = zip(*data_loc.apply(lambda row: katec_to_wgs84(row['x좌표'], row['y좌표']), axis=1))

In [21]:
# 잘 변환됐는지 확인
data_loc.head()

,격자ID,x좌표,y좌표,longitude,latitude
0,가다76b67a,776500,1567000,125.075837,34.075020
1,가다76b72a,776500,1572000,125.074553,34.120073
2,가다77a66b,777000,1566500,125.081380,34.070622
3,가다77a67a,777000,1567000,125.081252,34.075127
4,가다77a71b,777000,1571500,125.080098,34.115675


#### ② *카드매출_가맹점 > cell_id* 에 맵핑

In [23]:
# 카드매출_가맹점 > cell_id 종류
data_biz.cell_id.unique()

array(['다바88b09b', '다바88b10a', '다바89a09b', '다바89a10a', '다바89a10b',
       '다바89a11b', '다바89b09b', '다바89b10b', '다바89b11a', '다바89b11b',
       '다바89b12a', '다바89b12b', '다바89b13a', '다바89b13b', '다바90a10b',
       '다바90a11a', '다바90a11b', '다바90a12a', '다바90a12b', '다바90a13a',
       '다바90a13b', '다바90a14a', '다바90a14b', '다바90b08a', '다바90b08b',
       '다바90b10b', '다바90b11a', '다바90b12a', '다바90b12b', '다바90b13a',
       '다바90b13b', '다바90b14a', '다바90b14b', '다바90b15a', '다바91a11b',
       '다바91a12a', '다바91a12b', '다바91a13a', '다바91a13b', '다바91a14a',
       '다바91a14b', '다바91a15a', '다바91a15b', '다바91a16b', '다바91b11a',
       '다바91b12a', '다바91b12b', '다바91b13a', '다바91b13b', '다바91b14a',
       '다바91b14b', '다바91b15a', '다바91b15b', '다바91b16a', '다바91b16b',
       '다바92a12a', '다바92a12b', '다바92a13a', '다바92a13b', '다바92a14a',
       '다바92a14b', '다바92a15a', '다바92a15b', '다바92a16a', '다바92b04a',
       '다바92b12a', '다바92b12b', '다바92b13a', '다바92b13b', '다바92b14a',
       '다바92b14b', '다바92b15a', '다바92b15b', '다바93a12a', '다바93a1

- 다바8809b~다바93a05a 까지 중구라고 할 수 있음

</br>

- *국토지리정보원_국토조사_(격자)500M*에서 '다바'를 포함하고 있는 cell_id 찾아서
- *카드매출_가맹점 > cell_id*에 위도, 경도 맵핑

In [25]:
## 1) *국토지리정보원_국토조사_(격자)500M*에서 '다바'를 포함하고 있는 격자_id
junggu_loc = data_loc[data_loc['격자ID'].str.contains('다바')]


## 2) *카드매출_가맹점 > cell_id*와 *국토지리정보원_국토조사_(격자)500M >격자 ID* 비교하여 위/경도 컬럼 붙이기
def get_coordinates(cell_id):
    
    # junggu_loc에서 해당 cell_id를 가진 행을 찾음
    result = junggu_loc[junggu_loc['격자ID'] == cell_id]

    # 만약 해당 cell_id가 존재하면 위도와 경도를 반환하고, 없으면 None을 반환
    if not result.empty:
        return result.iloc[0]['latitude'], result.iloc[0]['longitude']
    else:
        return None, None

# 카드매출_가맹점에 새로운 컬럼으로 위도와 경도를 추가
data_biz['latitude'], data_biz['longitude'] = zip(*data_biz['cell_id'].apply(get_coordinates))

In [27]:
data_biz[['latitude', 'longitude']].isna().sum()

latitude     0
longitude    0
dtype: int64

### (2) 업종대분류, 업종중분류, 업종소분류

#### ① 결측치 : drop

In [29]:
# 결측치 확인
print(data_biz[['업종대분류', '업종중분류', '업종소분류']].isna().sum())
print(data_biz.shape)

업종대분류     15652
업종중분류     23604
업종소분류    178261
dtype: int64
(567495, 15)


***카드매출_가맹점* 을 통해서 업종소분류를 파악하는 것이 중요하기 때문에 업종소분류가 결측치인 데이터는 모두 제외**

In [31]:
# 결측치인 값은 제외
data_biz = data_biz[data_biz['업종소분류'].isna() == False]
data_biz.shape

(389234, 15)

In [33]:
# 결측치 처리 후 다시 확인
print(data_biz[['업종대분류', '업종중분류', '업종소분류']].isna().sum())
print(data_biz.shape)

업종대분류    0
업종중분류    0
업종소분류    0
dtype: int64
(389234, 15)


#### ② 업종분류명 컬럼 추가

##### **방법[1]** ***상가(상권)정보_업종분류.xlsx***
- 제공된 데이터 중 업종분류코드별 업종분류명이 표기돼 있는 데이터
- 그런데 해당 데이터는 ***카드매출_가맹점***에 있는 모든 업종분류코드를 설명하고 있지 않음.

In [35]:
# 카드매출_가맹점 > 업종대분류
print(f'카드매출_가맹점 > 업종대분류\n {data_biz.업종대분류.unique()}\n\n')

# 상가(상권)정보_업종분류 > 대분류코드
print(f'상가(상권)정보_업종분류 > 대분류코드\n {data_sort.대분류코드.unique()}')

카드매출_가맹점 > 업종대분류
 ['G' 'I' 'M' 'S' 'P' 'O' 'R' 'N' 'C' 'Q' 'H' 'J' 'K' 'L']


상가(상권)정보_업종분류 > 대분류코드
 ['G2' 'I1' 'I2' 'L1' 'M1' 'N1' 'P1' 'Q1' 'R1' 'S2']


- *상가(상권)정보_업종분류* 는 *카드매출_가맹점* 의 모든 업종분류를 포함하고 있지 않음  
- 따라서 업종분류명 컬럼을 추가할 때는 다음 방법[2]를 사용

##### **방법[2]** ***한국표준산업분류10차_표.xlsx***
- 외부 데이터인 ***통계청 한국표준산업분류10차***(한국표준산업분류10차_표.xlsx) 활용  
- 해당 데이터의 출처는 다음과 같음.  
  http://ecosmart.kaist.ac.kr/bbs/board.php?bo_table=menu3_1&wr_id=1

In [37]:
# 한국표준산업분류10차 데이터 정리

# 소분류, 세분류 컬럼 drop (카드매출_가맹점 > 업종소분류 == 한국표준산업분류10차 > 세세분류)
data_code.drop(['코드.2', '항목명.2', '코드.3', '항목명.3'], axis=1, inplace=True)

# 컬럼명 정리
data_code.columns = ['대분류_코드', '대분류_항목명',
                     '중분류_코드', '중분류_항목명',
                     '소분류_코드', '소분류_항목명']

# 결측치 채워넣기 (ex. 대분류코드 A ~ B 사이 결측치는 A로, B ~ C 사이 결측치는 B로)
data_code = data_code.ffill()

# 확인
print(f'대분류_코드 : {data_code['대분류_코드'].nunique()}')
print(f'대분류_항목명 : {data_code['대분류_항목명'].nunique()}')
print(f'중분류_코드 : {data_code['중분류_코드'].nunique()}')
print(f'중분류_항목명 : {data_code['중분류_항목명'].nunique()}')
print(f'소분류_코드 : {data_code['소분류_코드'].nunique()}')
print(f'소분류_항목명 : {data_code['소분류_항목명'].nunique()}')

대분류_코드 : 21
대분류_항목명 : 21
중분류_코드 : 77
중분류_항목명 : 77
소분류_코드 : 1196
소분류_항목명 : 1196


In [385]:
data_code.columns

Index(['대분류_코드', '대분류_항목명', '중분류_코드', '중분류_항목명', '소분류_코드', '소분류_항목명'], dtype='object')

In [39]:
# 카드매출_가맹정에 붙이기 위한 마지막 사전 작업

## 대분류_코드 - 대분류_항목명
code_big = data_code['대분류_코드'].unique().tolist()
code_name_big = data_code['대분류_항목명'].unique().tolist()

## 중분류_코드 - 중분류_항목명
code_mid = data_code['중분류_코드'].unique().tolist()
code_name_mid = data_code['중분류_항목명'].unique().tolist()

## 소분류_코드 - 소분류_항목명
code_small = data_code['소분류_코드'].unique().tolist()
code_name_small = data_code['소분류_항목명'].unique().tolist()



## 각각 딕셔너리로 만들기
dict_big = dict(zip(code_big, code_name_big))
dict_mid = dict(zip(code_mid, code_name_mid))
dict_small = dict(zip(code_small, code_name_small))

In [391]:
dict_big

{'A': '농업, 임업 및 어업(01~03)',
 'B': '광업(05~08)',
 'C': '제조업(10~34)',
 'D': '전기, 가스, 증기 및 공기 조절 공급업(35)',
 'E': '수도, 하수 및 폐기물 처리, 원료 재생업(36~39)',
 'F': '건설업(41~42)',
 'G': '도매 및 소매업(45~47)',
 'H': '운수 및 창고업(49~52)',
 'I': '숙박 및 음식점업(55~56)',
 'J': '정보통신업(58~63)',
 'K': '금융 및 보험업(64~66)',
 'L': '부동산업(68)',
 'M': '전문, 과학 및 기술 서비스업(70~73)',
 'N': '사업시설 관리, 사업 지원 및 임대 서비스업(74~76)',
 'O': '공공 행정, 국방 및 사회보장 행정(84)',
 'P': '교육 서비스업(85)',
 'Q': '보건업 및 사회복지 서비스업(86~87)',
 'R': '예술, 스포츠 및 여가관련 서비스업(90~91)',
 'S': '협회 및 단체, 수리 및 기타 개인 서비스업(94~96)',
 'T': '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98)',
 'U': '국제 및 외국기관(99)'}

In [41]:
# *카드매출_가맹점 > 업종소분류*는 *업종대분류*의 문자가 맨 앞에 붙음
# 이후 분석 시 문자형보다는 수치형이 좋기 때문에 *카드매출_가맹점 > 업종소분류* 맨앞에 붙어있는 알파벳을 제거

import re
def to_numeric(text):
    numbers = re.findall(r'\d+', text)
    return numbers[0]

data_biz['업종소분류'] = data_biz['업종소분류'].apply(to_numeric).astype(float)
data_biz

,기준일자,cell_id,업종대분류,업종중분류,업종소분류,이용건수,이용건수_지역화폐,이용건수_재난지원금,이용건수_기타,이용금액,이용금액_지역화폐,이용금액_재난지원금,이용금액_기타,latitude,longitude
1,20230701,다바88b09b,G,47.0,47212.0,8,1,0,7,1659200,207100,0,1452100,36.285486,127.369806
2,20230701,다바88b09b,G,47.0,47519.0,3,0,0,3,132000,0,0,132000,36.285486,127.369806
3,20230701,다바88b09b,G,47.0,47711.0,16,1,0,15,827338,70000,0,757338,36.285486,127.369806
4,20230701,다바88b09b,G,47.0,47712.0,11,1,0,10,312355,43983,0,268372,36.285486,127.369806
5,20230701,다바88b09b,I,56.0,56111.0,39,0,0,39,1421500,0,0,1421500,36.285486,127.369806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46293,20240630,다바95b11b,G,47.0,47513.0,1,0,0,1,5500,0,0,5500,36.303576,127.447743
46294,20240630,다바95b11b,G,47.0,47519.0,1,0,0,1,5000,0,0,5000,36.303576,127.447743
46295,20240630,다바95b11b,I,56.0,56111.0,21,0,0,21,626400,0,0,626400,36.303576,127.447743
46296,20240630,다바96a10b,I,56.0,56111.0,5,0,0,5,73600,0,0,73600,36.294563,127.453318


In [49]:
# *카드매출_가맹점*에 붙이기

## 새로운 변수에 할당
data_biz_cd = data_biz.copy()

data_biz_cd['업종대분류_항목명'] = data_biz_cd['업종대분류'].map(dict_big)
data_biz_cd['업종중분류_항목명'] = data_biz_cd['업종중분류'].map(dict_mid)
data_biz_cd['업종소분류_항목명'] = data_biz_cd['업종소분류'].map(dict_small)

## 컬럼 순서 변경
data_biz_cd = data_biz_cd[['기준일자', 'cell_id', 'latitude', 'longitude', '업종대분류', '업종대분류_항목명', '업종중분류', '업종중분류_항목명', '업종소분류', '업종소분류_항목명',
                       '이용건수', '이용건수_지역화폐', '이용건수_재난지원금', '이용건수_기타',
                       '이용금액', '이용금액_지역화폐', '이용금액_재난지원금', '이용금액_기타']]

##### **결측치**

In [51]:
# 업종분류명 붙인 후 결측치 확인
data_biz_cd[['업종대분류_항목명', '업종중분류_항목명', '업종소분류_항목명']].isna().sum()

업종대분류_항목명    0
업종중분류_항목명    0
업종소분류_항목명    0
dtype: int64

- 결측치 없음

### (3) 전처리 완료 데이터 저장

In [54]:
# 이후 카드매출_가맹점은 '카드매출(가맹점)_전처리_완료.csv' 데이터를 새로 불러와서 사용
data_biz_cd.to_csv(os.path.join(path, '카드매출(가맹점)_전처리_완료.csv'), index = False, encoding = 'cp949')

## 1-3 카드매출 (유입고객)

In [304]:
# 데이터 기본 정보 확인
data_cus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4849230 entries, 0 to 395655
Data columns (total 25 columns):
 #   Column      Dtype  
---  ------      -----  
 0   기준년월        int64  
 1   광역시_상권      int64  
 2   시군구_상권      int64  
 3   행정동_상권      float64
 4   업종대분류       object 
 5   업종중분류       float64
 6   업종소분류       object 
 7   업력          object 
 8   광역시_유입      float64
 9   시군구_유입      float64
 10  구분_휴일평일     int64  
 11  구분_시간대      int64  
 12  구분_개인법인     int64  
 13  연령          float64
 14  성별          object 
 15  추정소득구간      float64
 16  신용등급구간      float64
 17  이용건수        int64  
 18  이용건수_지역화폐   float64
 19  이용건수_재난지원금  float64
 20  이용건수_기타     float64
 21  이용금액        int64  
 22  이용금액_지역화폐   int64  
 23  이용금액_재난지원금  int64  
 24  이용금액_기타     int64  
dtypes: float64(10), int64(11), object(4)
memory usage: 961.9+ MB


In [306]:
# 데이터 결측치 확인
data_cus.isna().sum()

기준년월                0
광역시_상권              0
시군구_상권              0
행정동_상권         306956
업종대분류            1016
업종중분류          307965
업종소분류          307965
업력             306956
광역시_유입          73832
시군구_유입         289597
구분_휴일평일             0
구분_시간대              0
구분_개인법인             0
연령             204960
성별             204575
추정소득구간         215765
신용등급구간         215765
이용건수                0
이용건수_지역화폐     4162655
이용건수_재난지원금    4849230
이용건수_기타        286521
이용금액                0
이용금액_지역화폐           0
이용금액_재난지원금          0
이용금액_기타             0
dtype: int64

### (1) 시군구_상권, 광역시_상권 : drop

In [308]:
# 시군구 상권 데이터 종류
print(data_cus.시군구_상권.unique())
print(data_cus.광역시_상권.unique())

[30140]
[30]


- 데이터가 각각 30140과 30, 대전 중구와 대전광역시 하나씩 밖에 없기 때문에 해당 컬럼들은 drop해도 됨

In [311]:
# drop
data_cus.drop(['시군구_상권', '광역시_상권'], axis=1, inplace=True)

### (2) 업종대분류, 업종중분류, 업종소분류

#### ① 결측치 : drop

In [313]:
# 업종대분류가 결측치일 때, 업종중분류, 업종소분류도 모두 결측치인지
data_cus[data_cus.업종대분류.isna()]['업종중분류'].isna().sum() == data_cus[data_cus.업종대분류.isna()].shape[0]

True

In [314]:
# 업종중분류가 결측치일 때, 업종소분류도 결측치인지 (반대의 경우도)
data_cus[data_cus.업종중분류.isna()].equals(data_cus[data_cus.업종소분류.isna()])

True

In [316]:
# *행정동_상권*이 결측치일 때, *업종소분류*도 무조건 결측치인지 확인
data_cus[data_cus.업종소분류.isna()]['행정동_상권'].isna().sum() == data_cus[data_cus.행정동_상권.isna()].shape[0]

True

In [317]:
# 결측치 개수가 똑같았던 *업력*의 결측치 데이터와 일치하는지 비교
data_cus[data_cus.행정동_상권.isna()].equals(data_cus[data_cus['업력'].isna()])

True

**결측치 개수가 동일한 컬럼**
- 업종중분류 & 업종소분류
- 행정동_상권 & 업력

</br>

**결측치 데이터 간의 관계**
- 업종대분류가 결측치일 때, 업종중분류 & 업종소분류도 모두 결측치  
  → 업종중분류/업종소분류 결측치 ⊃ 업종대분류 결측치
- 행정동_상권 & 업력이 결측치일 때, 업종중분류 & 업종소분류도 모두 결측치  
  → 업종중분류/업종소분류 결측치 ⊃ 행정동_상권 & 업력 결측치
  
- 정리하면 업종중분류/업종소분류 결측치 ⊃ 업종대분류 결측치, 행정동_상권 & 업력 결측치
- 따라서 **업종중분류의 결측치 (=업종소분류의 결측치)만 지우면 관련 결측치가 한꺼번에 없어짐**

***카드매출_유입고객*을 통해서 업종소분류를 파악하는 것이 중요하기 때문에 업종소분류가 결측치인 데이터는 모두 제외**

In [321]:
# 업종중분류 결측치 (=업종소분류 결측치) 삭제
data_cus = data_cus[data_cus['업종중분류'].isna() == False]

#### ② 업종분류명 컬럼 추가
- *카드매출_가맹점*과 같은 방법으로 외부 데이터인 통계청 한국표준산업분류10차(한국표준산업분류10차_표.xlsx) 활용

In [323]:
# *카드매출_유입고객 > 업종소분류*는 *업종대분류*의 문자가 맨 앞에 붙음
# 이후 분석 시 문자형보다는 수치형이 좋기 때문에 *카드매출_유입고객 > 업종소분류* 맨앞에 붙어있는 알파벳을 제거

import re
def to_numeric(text):
    numbers = re.findall(r'\d+', text)
    return numbers[0]

data_cus['업종소분류'] = data_cus['업종소분류'].apply(to_numeric).astype(float)
data_cus.head()

,기준년월,행정동_상권,업종대분류,업종중분류,업종소분류,업력,광역시_유입,시군구_유입,구분_휴일평일,구분_시간대,...,추정소득구간,신용등급구간,이용건수,이용건수_지역화폐,이용건수_재난지원금,이용건수_기타,이용금액,이용금액_지역화폐,이용금액_재난지원금,이용금액_기타
24774,202307,30140535.0,C,14.0,14191.0,10년이하,NaN,NaN,1,3,...,9.0,9.0,1,NaN,NaN,1.0,49000,0,0,49000
24775,202307,30140535.0,C,14.0,14191.0,10년이하,NaN,NaN,2,2,...,9.0,9.0,1,NaN,NaN,1.0,147000,0,0,147000
24776,202307,30140535.0,C,14.0,14191.0,10년이하,11.0,NaN,1,4,...,NaN,NaN,1,NaN,NaN,1.0,34300,0,0,34300
24777,202307,30140535.0,C,14.0,14191.0,10년이하,11.0,NaN,2,4,...,NaN,NaN,1,NaN,NaN,1.0,36000,0,0,36000
24778,202307,30140535.0,C,14.0,14191.0,10년이하,11.0,11170.0,1,4,...,3.0,2.0,1,NaN,NaN,1.0,17000,0,0,17000


In [326]:
# 업종분류명 컬럼 추가

## 앞서 *카드매출_가맹점*에서 사용한 맵핑 딕셔너리 활용
## 사본 생성 
data_cus_cd = data_cus.copy()
data_cus_cd['업종대분류_항목명'] = data_cus_cd['업종대분류'].map(dict_big)
data_cus_cd['업종중분류_항목명'] = data_cus_cd['업종중분류'].map(dict_mid)
data_cus_cd['업종소분류_항목명'] = data_cus_cd['업종소분류'].map(dict_small)

In [327]:
# 결측치 확인
print(data_cus_cd[['업종대분류_항목명', '업종중분류_항목명', '업종소분류_항목명']].isna().sum())
print(data_cus_cd[data_cus_cd['업종소분류_항목명'].isna() == True]['업종중분류_항목명'].unique())

업종대분류_항목명    0
업종중분류_항목명    0
업종소분류_항목명    0
dtype: int64
[]


### (3) 행정동_상권 : 행정동명 컬럼 추가

In [330]:
# 맵핑할 리스트 생성
list_code_num = data_cus_cd['행정동_상권'].unique().tolist()
list_name = []

# 행정동 코드 str 타입으로 변환
data_admin['행정동코드'] = data_admin['행정동코드'].astype('str')

for i in data_cus_cd['행정동_상권'].unique().astype('str').tolist():
  list_name.append(data_admin[data_admin['행정동코드'].str.contains(i)].iloc[0]['읍면동명'])

# 딕셔너리 생성
cus_admin_cd = dict(zip(list_code_num, list_name))

# 컬럼 붙이기
data_cus_cd['행정동명'] = data_cus_cd['행정동_상권'].map(cus_admin_cd)

### (4) 시군구_유입, 추정소득구간, 신용등급구간 : 결측치 drop
- 해당 변수들은 결측치를 보간할 방법이 따로 없음- 이후 분석의 용이성을 위해 결측치는 모두 제외

In [332]:
# 결측치 확인
data_cus_cd[['시군구_유입', '추정소득구간', '신용등급구간']].isna().sum()

시군구_유입    266422
추정소득구간    194508
신용등급구간    194508
dtype: int64

In [334]:
# 시군구_유입이 결측일 때, 추정소득구간 & 신용등급구간도 결측인지
print(f'시군구_유입이 결측일 때, 추정소득구간도 결측인가 : {data_cus_cd[data_cus_cd['시군구_유입'].isna()]['추정소득구간'].isna().sum() == data_cus_cd['추정소득구간'].isna().sum()}')
print(f'추정소득구간 결측치와 신용등급구간 결측치가 일치하는가 : {data_cus_cd['추정소득구간'].isna().sum() == data_cus_cd['신용등급구간'].isna().sum()}')

시군구_유입이 결측일 때, 추정소득구간도 결측인가 : True
추정소득구간 결측치와 신용등급구간 결측치가 일치하는가 : True


In [336]:
# 결측치 drop 
data_cus_cd = data_cus_cd[data_cus_cd['시군구_유입'].isna() == False]

### (5) 연령, 성별 : 결측치 drop
- 해당 변수들 역시 결측치를 보간할 방법이 따로 없음
- 이후 분석에서 연령과 성별을 파악하는 것이 중요하기 때문에 결측치인 데이터는 모두 제외

In [338]:
# 결측치 확인
data_cus_cd[['연령', '성별']].isna().sum()

연령    117208
성별    117208
dtype: int64

In [340]:
# 결측치 없애기
data_cus_cd = data_cus_cd[(data_cus_cd['연령'].isna() == False) & (data_cus_cd['성별'].isna() == False)]

### (6) 이용건수
- 이용건수, 이용건수_지역화폐, 이용건수_재난지원금, 이용건수_기타

#### ① 이용건수_지역화폐, 이용건수_재난지원금, 이용건수_기타 : 결측치 0으로 채우기

- 결측치를 0으로 채웠을 때 다음과 같은 등식이 성립하는 지 확인  
→ **이용건수 = 이용건수_지역화폐 + 이용건수_재난지원금 + 이용건수_기타**

In [342]:
dcd = data_cus_cd.copy()
dcd[['이용건수_지역화폐', '이용건수_재난지원금', '이용건수_기타']] = dcd[['이용건수_지역화폐', '이용건수_재난지원금', '이용건수_기타']].fillna(0)
sum((dcd['이용건수'] == dcd['이용건수_지역화폐'] + dcd['이용건수_재난지원금'] + dcd['이용건수_기타']) == False)

0

- 즉, *이용건수_지역화폐, 이용건수_재난지원금, 이용건수_기타* 의 결측치는 0으로 채워야 함

In [344]:
data_cus_cd[['이용건수_지역화폐', '이용건수_재난지원금', '이용건수_기타']] = data_cus_cd[['이용건수_지역화폐', '이용건수_재난지원금', '이용건수_기타']].fillna(0)

#### ② 이용건수_재난지원금 : drop
- 모든 행이 결측치이므로 컬럼 자체를 삭재

In [346]:
# 이용건수_재난지원금 == 0인 값과 카드매출_유입고객 데이터의 길이가 같은지 확인
# 즉, 모든 행이 0인지 확인
sum((data_cus_cd['이용건수_재난지원금']==0) == True) == data_cus_cd.shape[0]

True

In [348]:
# 핻 삭제
data_cus_cd.drop('이용건수_재난지원금', axis=1, inplace=True)

In [349]:
data_cus_cd.isna().sum()

기준년월          0
행정동_상권        0
업종대분류         0
업종중분류         0
업종소분류         0
업력            0
광역시_유입        0
시군구_유입        0
구분_휴일평일       0
구분_시간대        0
구분_개인법인       0
연령            0
성별            0
추정소득구간        0
신용등급구간        0
이용건수          0
이용건수_지역화폐     0
이용건수_기타       0
이용금액          0
이용금액_지역화폐     0
이용금액_재난지원금    0
이용금액_기타       0
업종대분류_항목명     0
업종중분류_항목명     0
업종소분류_항목명     0
행정동명          0
dtype: int64

In [356]:
# 컬럼 순서 변경
data_cus_cd = data_cus_cd[['기준년월', '행정동_상권', '행정동명', 
                           '업종대분류', '업종대분류_항목명', '업종중분류', '업종중분류_항목명', '업종소분류', '업종소분류_항목명', 
                           '업력', '광역시_유입', '시군구_유입', '구분_휴일평일', '구분_시간대', '구분_개인법인', '연령', '성별', '추정소득구간', '신용등급구간', 
                           '이용건수','이용건수_기타', '이용건수_지역화폐', '이용금액', '이용금액_기타','이용금액_지역화폐']]

### (7) 전처리 완료 데이터 저장

In [358]:
# 이후 카드매출_유입고객 데이터는 '카드매출(유입고객)_전처리_완료.csv'를 사용
data_cus_cd.to_csv(os.path.join(path, '카드매출(유입고객)_전처리_완료.csv'), index=False, encoding = 'cp949')

# 2 이동경로 분석

In [5]:
# 전처리 완료된 유동인구 데이터 불러오기
data = pd.read_csv(os.path.join(path, '유동인구_전처리_완료.csv'), encoding='cp949')

## 2-1 성심당 위치 파악
- *유동인구_전처리_완료* 데이터 중 위치를 나타내는 *id, latitude, longitude*만 불러오기
- *유동인구_전처리_완료 > latitude, longitude* 중 성심당 위도, 경도 ± 50m 반경에 있는 좌표 찾기
- 성심당 위도, 경도 중심점으로 잡아서 가장 가까운 *latitude, longitude* 값 추출

In [7]:
# 구글맵 기준 성심당의 좌표를 기준으로 변수 설정
sungsimdang_lat = 36.32765 # 성심당의 위도
sungsimdang_lon = 127.4273  # 성심당의 경도

# 50m의 범위를 대략적으로 위도와 경도로 변환 (1도는 약 111km)
lat_range = 0.00045  # 위도의 범위
lon_range = 0.00045  # 경도의 범위

# 범위 설정
min_lat = sungsimdang_lat - lat_range
max_lat = sungsimdang_lat + lat_range
min_lon = sungsimdang_lon - lon_range
max_lon = sungsimdang_lon + lon_range

# 범위를 이용해 데이터 필터링
df_bread10 = data[(data['latitude'] >= min_lat) & (data['latitude'] <= max_lat) &
                    (data['longitude'] >= min_lon) & (data['longitude'] <= max_lon)]

df_bread10 = df_bread10[['id', 'latitude', 'longitude']]

In [9]:
# 성심당을 포함하는 50m cell 후보지
df_bread10.id.unique()

array([60123258, 60111069, 60123259, 60111070], dtype=int64)

In [11]:
df_bread10['distance_to_centroid'] = ((df_bread10['latitude'] - sungsimdang_lat)**2 + 
                                      (df_bread10['longitude'] - sungsimdang_lon)**2)**0.5

# 중심점에서 가장 가까운 id 찾기
df_closest_to_centroid = df_bread10.loc[[df_bread10['distance_to_centroid'].idxmin()]]
df_closest_to_centroid

,id,latitude,longitude,distance_to_centroid
2494666,60123258,36.327457,127.42757,0.000332


**→ 성심당 유동인구를 나타낼 수 있는 cell은 '60123258'번 cell**

## 2-2 이동경로

### (1) 이동경로 분석 전, 필요한 변수 정리
- SUNG_SIM_DANG_ID : 성심당 cell id
- SUNG_SIM_DANG_OPENING_HOUR & SUNG_SIM_DANG_CLOSING_HOUR : 성심당 영업시간
  - SUNG_SIM_DANG_OPENING_HOUR = 8
  - SUNG_SIM_DANG_CLOSING_HOUR = 21 : 본래 영업종료 시간이 22시이나, timeze_cd == 21는 21시 ~ 21시 59분 1시간 동안의 유동인구를 의미하는 것이기 때문에 21시로 지정
- MAX_DISTANCE_KM : 1시간동안 한 사람이 이동할 수 있는 최대 거리
- COSINE_SIMILARITY_THRESHOLD : 코사인 유사도 임계값 (임계값을 넘는 위치를 다음 이동 장소 후보로 지정)

In [11]:
# 성심당 cell id
SUNG_SIM_DANG_ID = 60123258

# 성심당 영업시간
SUNG_SIM_DANG_OPENING_HOUR = 8
SUNG_SIM_DANG_CLOSING_HOUR = 21

# 1시간동안 한 사람이 이동할 수 있는 최대 거리
MAX_DISTANCE_KM = 3

# 코사인 유사도 임계값
COSINE_SIMILARITY_THRESHOLD = 0.8

# 연령/성별 맵핑 딕셔너리
age_gender_mapping = {
    'm00': ('10세 미만', '남성'),
    'm10': ('10대', '남성'),
    'm20': ('20대', '남성'),
    'm30': ('30대', '남성'),
    'm40': ('40대', '남성'),
    'm50': ('50대', '남성'),
    'm60': ('60대', '남성'),
    'm70': ('70대이상', '남성'),
   
    'f00': ('10세 미만', '여성'),
    'f10': ('10대', '여성'),
    'f20': ('20대', '여성'),
    'f30': ('30대', '여성'),
    'f40': ('40대', '여성'),
    'f50': ('50대', '여성'),
    'f60': ('60대', '여성'),
    'f70': ('70대이상', '여성')
}


# 성심당 유동인구 필터링
sungsimdang_visits = data[
    (data['id'] == SUNG_SIM_DANG_ID) &
    (data['timezn_cd'] >= SUNG_SIM_DANG_OPENING_HOUR) &
    (data['timezn_cd'] <= SUNG_SIM_DANG_CLOSING_HOUR)
]

### (2) 거리 계산 함수 : geodesic 패키지 활용
- geodesic 패키지를 사용하여 거리를 km로 반환
- geodesic이 Numba와 호환되지 않기 때문에 Numba 사용X

In [13]:
def calculate_distance(coord1_lat, coord1_lon, coord2_lat, coord2_lon):
    return geodesic((coord1_lat, coord1_lon), (coord2_lat, coord2_lon)).kilometers

### (3) 코사인 유사도 계산 함수
- 코사인 유사도 계산 공식을 함수화
- numba의 njit : Python의 동적 기능을 배제하고 네이티브 코드로 컴파일하여 실행 속도가 크게 향상

In [16]:
@njit
def calculate_cosine_similarity(lat1, lon1, pop1, lat2, lon2, pop2):
    vec1 = np.array([lat1, lon1, pop1])
    vec2 = np.array([lat2, lon2, pop2])
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

### (4) 성심당 방문 전후 경로 추적 함수
- 성심당 방문시간별로 전후 방문 경로 추적  
  - 예시1) 성심당 12시 방문 → 0 ~ 11시 & 13시 ~ 23시 성심당 외 방문 장소 추적  
  - 예시2) 성심당 13시 방문 → 0 ~ 12시 & 14시 ~ 23시 성심당 외 방문 장소 추적
- 성심당 방문시간 전후 총 23시간의 방문 장소 추출
- 즉, 성심당 영업시간대 14개(8시~21시) * 23시간 동안의 방문 장소를 추적  
  → (14 * 23)개의 행 생성
- **자세한 설명은 README.md 참조**

In [19]:
# 함수 정의
def track_movement_for_day(date, daily_data, age_gender_col, sungsimdang_visits):

    # 반환용 리스트 생성
    movement_data = []

    # 성심당 유동인구 데이터를 날짜별로 추출
    # date : (6)에서 groupby한 date
    sungsimdang_visits_on_date = sungsimdang_visits[sungsimdang_visits['etl_ymd'] == date]

    # 성심당 방문 전/후 이동경로 추출
    # 코사인 유사도 임계값을 넘고 최대 이동 거리를 넘지 않는 위치 
    def trace_route(visit_row, direction='before'):

        # 유동인구가 유효한 경우만을 저장할 리스트
        movement_data_local = []

        '''
        - 성심당을 방문한 시간을 현재 방문장소로 설정하고 전후 이동경로 분석
        - 성심당 방문 데이터는 성심당 영업시간 동안의 유동인구 데이터
        - 즉 앞서 (1)에서 정의한 8 ~ 21시의 성심당 유동인구 데이터
        '''
        previous_visit = visit_row

        
        ''' 
        ===
        1) 성심당 방문 '전' 이동경로는 성심당 방문 시간을 시작으로 역순으로 방문장소 추적 → range(visit_row['timezn_cd'] - 1, -1, -1)
           e.g. << (성심당 12시 방문) → 11시 방문 장소 → 10시 방문 장소 → ``` → 0시 방문 장소 >> 

        === 
        2) 성심당 방문 '후' 이동경로는 성심당 방문 시간을 시작으로 순서대로 방문장소 추적 → range(visit_row['timezn_cd'] + 1, 24)
           e.g. << (성심당 12시 방문) → 13시 방문 장소 → 14시 방문 장소 → ``` → 23시 방문 장소 >> 
           
        '''
        valid_range = range(visit_row['timezn_cd'] - 1, -1, -1) if direction == 'before' else range(visit_row['timezn_cd'] + 1, 24)
        

        
        '''
        << 성심당 방문 '전' 이동경로>> (성심당 방문 '후' 이동경로는 해당X)
        
        - "이동거리(km)" 컬럼을 추가할 예정인데, 해당 컬럼는 "이전 장소와의 거리"를 나타냄
        - 현재 메커니즘으로 계산하면 "t-1시점 방문장소와 t-2시점 방문장소 간의 거리"가 "t-2시점 방문장소 행"에 추가됨
        - 즉, 이전 장소와의 거리가 아니라 "다음 장소와의 거리가 거리"가 기록됨
        - 따라서 "t-1시점 방문장소와 t-2시점 방문장소 간의 거리"는 "t-1시점 방문장소 행"에 기록될 수 있게 "이동거리(km)" 값은 리스트에 저장했다가 후처리할 예정
        '''
        prev_next_distance = []


        # 성심당 방문 전 시간대 범위 or 성심당 방문 후 시간대 범위
        for time in valid_range:
            time_visits = daily_data[daily_data['timezn_cd'] == time]

            # 시간대가 날짜별로 그룹화한 데이터에 없는 경우 pass
            if time_visits.empty:
                continue

            # (2)에서 정의한 *거리 계산 함수*로 이전 시간대 방문 장소와 현재 시간대의 모든 경위도 간의 거리 계산
            time_visits['distance'] = time_visits.apply(lambda row: calculate_distance(
                previous_visit['latitude'], previous_visit['longitude'], row['latitude'], row['longitude']
            ), axis=1)

            # 계산한 거리 중 (1)에서 정의한 MAX_DISTANCE_KM == 3km 를 넘지 않는 값만 필터링
            valid_visits = time_visits[time_visits['distance'] <= MAX_DISTANCE_KM]
            if valid_visits.empty:
                continue

            # 이전 시간대 방문장소와 현재 시간대 장소들간의 코사인 유사도 계산            
            valid_visits['cosine_similarity'] = valid_visits.apply(lambda row: calculate_cosine_similarity(
                previous_visit['latitude'], previous_visit['longitude'], previous_visit[age_gender_col],
                row['latitude'], row['longitude'], row[age_gender_col]
            ), axis=1)

            # 그 중 코사인 유사도 임계값(0.8)을 넘으면서 코사인 유사도 값이 가장 큰 장소 정하기
            most_similar_visit = valid_visits[valid_visits['cosine_similarity'] >= COSINE_SIMILARITY_THRESHOLD].nlargest(1, 'cosine_similarity')

            # 성심당 방문 "전" 이동경로의 이동거리는 앞서 정의한 prev_next_distance 리스트에 추가 후 나중에 처리
            if not most_similar_visit.empty:
                prev_next_distance.append(most_similar_visit.iloc[0]['distance'])
               
                movement_data_local.append({
                    '날짜': visit_row['etl_ymd'],
                    '공휴일': visit_row['공휴일'],
                    '주말': visit_row['주말'],
                    '연령대': age_gender_mapping[age_gender_col][0],
                    '성별': age_gender_mapping[age_gender_col][1],
                    '성심당_유동인구': visit_row[age_gender_col],
                    '성심당_방문시간': visit_row['timezn_cd'],
                    '전후_방문시간대': time,
                    '전후_방문장소_위도': most_similar_visit.iloc[0]['latitude'],
                    '전후_방문장소_경도': most_similar_visit.iloc[0]['longitude'],
                    '전후_방문장소_행정동명': most_similar_visit.iloc[0]['행정동명'],

                    # 성심당 방문 "전" 이동경로의 경우, 후처리 예정이기 때문에 None으로 비워두기
                    '이동거리(km)': None if direction == 'before' else prev_next_distance[-1]
                })
               
                previous_visit = most_similar_visit.iloc[0]

        '''
        성심당 방문 "전" 이동경로의 경우, prev_next_distance 리스트에 저장된 값을 다음 시간대의 "이동경로(km)" 컬럼에 삽입
        다시 말해, prev_next_distance 리스트를 정의할 때 설명한 바와 같이 "t-1시점 방문장소와 t-2시점 방문장소 간의 거리"는 "t-1시점 방문장소 행"에 기록
        '''
        if direction == 'before':
            prev_next_distance.append('-')
            for i in range(len(movement_data_local)):
                movement_data_local[i]['이동거리(km)'] = prev_next_distance[i + 1]

        
        movement_data_local = sorted(movement_data_local, key=lambda x: x['전후_방문시간대'])
        return movement_data_local




    # 해당날짜의 성심당 방문 데이터 (8 ~ 21시의 성심당 유동인구 데이터)
    for _, visit_row in sungsimdang_visits_on_date.iterrows():

        # 성심당 유동인구가 0인 경우, 전후 방문장소를 알 수 없기 때문에 '-'으로 처리
        if visit_row[age_gender_col] == 0:
            movement_data.append({
                '날짜': visit_row['etl_ymd'],
                '공휴일': visit_row['공휴일'],
                '주말': visit_row['주말'],
                '연령대': age_gender_mapping[age_gender_col][0],
                '성별': age_gender_mapping[age_gender_col][1],
                '성심당_유동인구': visit_row[age_gender_col],
                '성심당_방문시간': visit_row['timezn_cd'],
                '전후_방문시간대': '-',
                '전후_방문장소_위도': '-',
                '전후_방문장소_경도': '-',
                '전후_방문장소_행정동명': '-',
                '이동거리(km)': '-'
            })
            
        else:
            # 전/후 이동경로를 각각 구하기 위해 함수에 'before', 'after'을 설정
            movement_data.extend(trace_route(visit_row, 'before'))
            movement_data.extend(trace_route(visit_row, 'after'))

    return movement_data

### (5) 병렬 처리를 적용할 함수
- 날짜별 > 집단(연령/성별)별 로 같은 작업을 수행

In [22]:
# 병렬처리는 날짜별로만 되도록, 집단별 이동경로 구하는 함수 정의
def process_date_group(date, daily_data, sungsimdang_visits):

    # 집단별 이동경로를 저장할 리스트
    movement_data = []

    # 집단별로 이동경로 따로 구하기
    for age_gender_col in age_gender_mapping:
        movement_data.extend(track_movement_for_day(date, daily_data, age_gender_col, sungsimdang_visits))
    return movement_data

### (6) 정의한 함수를 병렬 처리
- 유동인구 데이터(원본)과 유동인구_전처리_완료 데이터 둘 다 용량이 10GB가 넘음
- 날짜별로 같은 작업을 수행하기 때문에 모든 날짜를 하나씩 처리하기보다는 병렬 처리
- 이를 위해 **jolbib** 라이브러리의 **Parallel, delayed** 클래스를 활용

In [26]:
# 날짜별로 데이터 그룹화
grouped_by_date = data.groupby('etl_ymd')

# 날짜별로 병렬 처리
movement_data = Parallel(n_jobs=-1)(delayed(process_date_group)(date, daily_data, sungsimdang_visits)
                                    for date, daily_data in grouped_by_date)

# 병렬 처리 후 결과를 flatten하여 통합
movement_data = [item for sublist in movement_data for item in sublist] 

# 결과를 데이터프레임으로 변환
df_movement = pd.DataFrame(movement_data)

# 결과 출력
df_movement

,날짜,공휴일,주말,연령대,성별,성심당_유동인구,성심당_방문시간,전후_방문시간대,전후_방문장소_위도,전후_방문장소_경도,전후_방문장소_행정동명,이동거리(km)
0,2023-08-12,공휴일 아님,주말,20대,여성,0.00,8,-,-,-,-,-
1,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,0,36.319641,127.395887,유천2동,-
2,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,1,36.319641,127.395887,유천2동,0.0
3,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,2,36.315546,127.388123,유천1동,0.832213
4,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,3,36.317396,127.397574,유천2동,0.873262
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,18,36.320633,127.414255,문화1동,1.414298
296,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,19,36.329262,127.428114,은행선화동,1.570126
297,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,20,36.32746,127.428127,은행선화동,0.200012
298,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,22,36.325183,127.423131,대흥동,0.471728


# 3 소비패턴 분석

## 3-1 데이터 전처리

In [28]:
# data 불러오기
industry_data = pd.read_csv(os.path.join(path, "카드매출(가맹점)_전처리_완료.csv"), encoding = 'cp949')
customer_data = pd.read_csv(os.path.join(path, "카드매출(유입고객)_전처리_완료.csv"), encoding = 'cp949')


# datetime type으로 변환
df_movement['날짜'] = pd.to_datetime(df_movement['날짜'])
industry_data['기준일자'] = pd.to_datetime(industry_data['기준일자'], format='%Y%m%d')
customer_data['기준년월'] = pd.to_datetime(customer_data['기준년월'], format='%Y%m')


# 공공 행정 등을 포함하는 업종대분류 O는 제외 (우리의 타겟층이 아님)
industry_data = industry_data[industry_data['업종대분류'] != 'O']
customer_data = customer_data[customer_data['업종대분류'] != 'O']

## 3-2 컬럼 추가
- 이동경로에 소비패턴 데이터를 이어가기 위해 *카드매출_유입고객*과 같은 형식의 컬럼 생성
  
- 이동경로 데이터에 추가한 컬럼 : 구분_시간대, 연령, 성별_숫자, 구분_휴일평일, 업력_숫자
- 이에 상응하는 *카드매출_유입고객* 컬럼 : 구분_시간대, 연령, 성별, 구분_휴일평일, 업력

In [30]:
## (1) 시간대 구분 함수
def get_time_period(hour):
    if 6 <= hour < 12:
        return 1
    elif 12 <= hour < 15:
        return 2
    elif 15 <= hour < 18:
        return 3
    elif 18 <= hour < 22:
        return 4
    else:
        return 5

df_movement['구분_시간대'] = df_movement['전후_방문시간대'].apply(lambda x: get_time_period(int(x)) if x != '-' else '-')



## (2) 연령대 변환 함수
def age(data):
    if data == '10세 미만':
        return 0
    elif data == '10대':
        return 1
    elif data == '20대':
        return 2
    elif data == '30대':
        return 3
    elif data == '40대':
        return 4
    elif data == '50대':
        return 5
    elif data == '60대':
        return 6
    else:
        return 6

df_movement['연령'] = df_movement['연령대'].apply(age)


## (3) 성별 변환
def gender(data):
    return 0 if data == '남성' else 1

df_movement['성별_숫자'] = df_movement['성별'].apply(gender)
customer_data['성별_numeric'] = customer_data['성별'].map({'M': 0, 'F': 1}).astype(np.int8)



## (4) 공휴일 평일 구분
def holiday(holi, weekend):
    return 1 if holi == '공휴일' or weekend == '주말' else 2

df_movement['구분_휴일평일'] = df_movement.apply(lambda row: holiday(row['공휴일'], row['주말']), axis=1)



## (5) 업력 : 수치형으로 변환
years = {
    '1년이하': 1,
    '3년이하': 3,
    '5년이하': 5,
    '10년이하': 10,
    '10년초과': 15
}
customer_data['업력_숫자'] = customer_data['업력'].map(years)


## (6) 날짜_숫자형
df_movement['날짜_숫자형'] = df_movement['날짜'].dt.strftime('%Y%m%d').astype(int)
industry_data['기준일자_숫자형'] = industry_data['기준일자'].dt.strftime('%Y%m%d').astype(int)
customer_data['기준년월_numeric'] = customer_data['기준년월'].dt.strftime('%Y%m').apply(lambda x: int(str(x)[4:6])).astype(np.int8)

## 3-3 이동경로 주변 업종 파악 : *카드매출_가맹점*을 통해
- **중요 가정** : *카드매출_가맹점*의 셀은 500m X 500m 격자의 중심점이다.  
    이는 ***1. 데이터 전처리*** 에서 시각화한 것과 같이 cell_id 간의 간격이 균등하다는 것에 근거함

### (1) 각 cell_id 중심점에서의 위도, 경도 범위 계산 함수
- 이동경로와 마찬가지로 Numba 적용하여 처리 속도 개선
- 이동경로에 사용된 *유동인구*데이터는 cell의 면적이 50m x 50m인 반면, *카드매출_가맹점*은 cell의 면적이 500m x 500m
- 따라서 *유동인구*의 셀을 포함하는 *카드매출_가맹점*의 셀을 필터링

In [32]:
# 거리 기반 계산 함수 - Numba 적용 (Numpy로 변환 후 계산)
@njit
def convert_cell_numba(lat, lon, cell_lat_array, cell_lon_array):
    lat_range = 0.00225
    lon_range = 0.00225
   
    lat_check = (cell_lat_array - lat_range <= lat) & (lat <= cell_lat_array + lat_range)
    lon_check = (cell_lon_array - lon_range <= lon) & (lon <= cell_lon_array + lon_range)

    return lat_check & lon_check

### (2) ***이동경로***에 ***카드매출_가맹점 > 업종소분류*** 를 맵핑하는 함수
- (1) 함수를 통해 *이동경로의 위치*가 *카드매출_가맹점의 어떤 cell*에 해당되는지 찾기
- 해당되는 cell에 기록되어있는 *카드매출_가맹점 > 업종소분류_코드, 업종소분류_항목명, 이용건수, 이용금액, 이용금액_지역화폐* 를 *이동경로* 데이터에 컬럼으로 붙이기
- 왼쪽 : *카드매출_가맹점*에서의 컬럼명 | 오른쪽 : *이동경로* 데이터에 추가한 컬럼명
  - 업종소분류 : 업종소분류_코드
  - 업종소분류_항목명 : 업종소분류_항목명
  - 이용건수 : 업종_일판매건수
  - 이용금액 : 업종_일매출
  - 이용금액_지역화폐 : 업종_일매출_지역화폐

In [35]:
def process_date_group(date, daily_data, industry_data):
    results = []
   
    for idx, row in daily_data.iterrows():
        
        # df_movement > 성심당_유동인구가 0인 경우
        if row['전후_방문장소_위도'] == '-':
            combined_row = row.to_dict()
            combined_row.update({
                '업종소분류_코드': '-',
                '업종소분류_항목명': '-',
                '업종_일판매건수': '-',
                '업종_일매출': '-',
                '업종_일매출_지역화폐': '-'
            })
            results.append(combined_row)
        
        # 유동인구가 0이 아닌 경우
        else:
            # 카드매출_가맹점 > latitude, longitude를 Numpy 배열로 변환
            cell_lat_array = industry_data['latitude'].to_numpy()
            cell_lon_array = industry_data['longitude'].to_numpy()
            lat = np.float64(row['전후_방문장소_위도']) 
            lon = np.float64(row['전후_방문장소_경도']) 

            # 카드매출_가맹점에 (1)의 계산 함수 적용하여 이동경로가 포함되는 cell 찾기
            matching_cells_bool = convert_cell_numba(lat, lon, cell_lat_array, cell_lon_array)
            matching_cells = industry_data[matching_cells_bool]
            matching_cells2 = matching_cells[matching_cells['기준일자_숫자형'] == row['날짜_숫자형']]

            # 위에서 필터링된 데이터의 행이 비어있을 경우, 이는 카드매출_가맹점에 존재하지 않는 값으로 판단
            # '주변 상권X' 으로 표시    e.g. 아파트 단지 등
            if matching_cells2.empty:
                combined_row = row.to_dict()
                combined_row.update({
                    '업종소분류_코드': '주변 상권X',
                    '업종소분류_항목명': '주변 상권X',
                    '업종_일판매건수': '주변 상권X',
                    '업종_일매출': '주변 상권X',
                    '업종_일매출_지역화폐': '주변 상권X'
                })
                results.append(combined_row)

  
            # 카드매출_가맹점 > 업종소분류, 업종소분류_항목명, 이용건수, 이용금액, 이용금액_지역화폐 컬럼의 값들을 각 행에 맞게 매칭
            else:
                for _, cell_row in matching_cells2.iterrows():
                    combined_row = row.to_dict()
                    combined_row.update({
                        '업종소분류_코드': cell_row['업종소분류'],
                        '업종소분류_항목명': cell_row['업종소분류_항목명'],
                        '업종_일판매건수': cell_row['이용건수'],
                        '업종_일매출': cell_row['이용금액'],
                        '업종_일매출_지역화폐': cell_row['이용금액_지역화폐']
                    })
                    results.append(combined_row)
                   
    return results

### (3) 정의한 함수를 날짜별로 병렬 처리

In [38]:
# 날짜별로 데이터를 그룹화
grouped_by_date = df_movement.groupby('날짜')

# 각 날짜별로 병렬 처리 (각 날짜 단위로 처리)
results_parallel = Parallel(n_jobs=-1)(delayed(process_date_group)(date, daily_data, industry_data)
                                      for date, daily_data in grouped_by_date)

# 중첩된 리스트(flatten) 처리
flattened_results = [item for sublist in results_parallel for item in sublist]

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(flattened_results)

# 카드매출_가맹점을 통해 추가된 컬럼을 제외한 컬럼들을 인덱스로 지정
# 한 방문장소당 여러 업종이 붙어있기 때문에 이후 편리한 처리를 위해 멀티인덱스화
result_df.set_index(df_movement.columns.to_list(), inplace=True)

# 결과 출력
result_df

업종소분류_코드  \
날짜         공휴일    주말 연령대 성별 성심당_유동인구 성심당_방문시간 전후_방문시간대 전후_방문장소_위도 전후_방문장소_경도 전후_방문장소_행정동명 이동거리(km) 구분_시간대 연령 성별_숫자 구분_휴일평일 날짜_숫자형              
2023-08-12 공휴일 아님 주말 20대 여성 0.00     8        -        -          -          -            -        -      2  1     1       20230812        -   
                            0.80     9        0        36.319641  127.395887 유천2동         -        5      2  1     1       20230812  47119.0   
                                                                                                                           20230812  47121.0   
                                                                                                                           20230812  47122.0   
                                                                                                                           20230812  47129.0   
...                                                                                                                                      ...   
                            30.92    21       18       36.320633  127.414255 문화1동         1.414298 4      2  1     1       20230812  96912.0   
                                              19       36.329262  127.428114 은행선화동        1.570126 4      2  1     1       20230812   주변 상권X   
                                              20       36.32746   127.428127 은행선화동        0.200012 4      2  1     1       20230812   주변 상권X   
                                              22       36.325183  127.423131 대흥동          0.471728 5      2  1     1       20230812   주변 상권X   
                                              23       36.320622  127.412028 문화1동         1.118101 5      2  1     1       20230812   주변 상권X   

                                                                                                                                              업종소분류_항목명  \
날짜         공휴일    주말 연령대 성별 성심당_유동인구 성심당_방문시간 전후_방문시간대 전후_방문장소_위도 전후_방문장소_경도 전후_방문장소_행정동명 이동거리(km) 구분_시간대 연령 성별_숫자 구분_휴일평일 날짜_숫자형                         
2023-08-12 공휴일 아님 주말 20대 여성 0.00     8        -        -          -          -            -        -      2  1     1       20230812                   -   
                            0.80     9        0        36.319641  127.395887 유천2동         -        5      2  1     1       20230812        기타 대형 종합 소매업   
                                                                                                                           20230812                슈퍼마켓   
                                                                                                                           20230812             체인화 편의점   
                                                                                                                           20230812  기타 음ㆍ식료품 위주 종합 소매업   
...                                                                                                                                                 ...   
                            30.92    21       18       36.320633  127.414255 문화1동         1.414298 4      2  1     1       20230812             가정용 세탁업   
                                              19       36.329262  127.428114 은행선화동        1.570126 4      2  1     1       20230812              주변 상권X   
                                              20       36.32746   127.428127 은행선화동        0.200012 4      2  1     1       20230812              주변 상권X   
                                              22       36.325183  127.423131 대흥동          0.471728 5      2  1     1       20230812              주변 상권X   
                                              23       36.320622  127.412028 문화1동         1.118101 5      2  1     1       20230812              주변 상권X   

                                                                                                                                    업종_일판매건수  \
날짜         공휴일    주말 연령대 성별 성심당_유동인구 성심당_방문시간 전후_방문시간대 전후_방문장소_위도 전후_방문장소_경도 전후_방문장소_행정동명 이동거리(km) 

## 3-4 ***이동경로*** 주변 업종 중 **해당 집단의 선호 업종** 파악 : *카드매출_유입고객*을 통해
- *카드매출_가맹점*으로 붙인 이동경로 주변 업종 중 해당 집단이 선호하는 업종을 파악하기 위해 *카드매출_유입고객*데이터 활용
- 업종을 단순 정렬하는 것이 아닌 **<< 연령 & 성별 & 방문 시간대 & 휴일/평일 & 날짜 >>** 별 유동인구 집단이 자주 가는 업종을 *이용건수 > 이용금액*을 기준으로 내림차순
- 위의 조건으로 *카드매출_유입고객*을 필터링했을 때 출력된 선호 업종과 일치하지 않는 이동경로 주변 업종은 "선호 업종X" 처리
- 단, 10세 미만의 경우 *카드매출_유입고객*에 기록된 데이터가 없기 때문에 '-' 처리

### (1) 추가할 컬럼을 위한 계산 함수
- 수치 연산을 최적화하기 위해 Numba 적용
- 추가할 컬럼 : 평균_업력, 고객_타지역_비율, 고객_월이용건수, 고객_월이용금액
- 각각 다음과 같이 계산 (데이터는 (2)에서 필터링될 예정)
  - 평균_업력 : *필터링된 데이터 > 업력* 의 평균
  - 고객_타지역_비율 : *필터링된 데이터 > 광역시_유입* 이 30이 아닌, 즉 대전광역시가 아닌 데이터의 비율
  - 고객_월이용건수 : *필터링된 데이터 > 이용건수*의 합
  - 고객_월이용금액 : *필터링된 데이터 > 이용금액*의 합

In [40]:
@njit
def calculate_customer_stats(dd_years, dd_area, dd_area_ndj, dd_count, dd_amount):
    years_mean = np.mean(dd_years)
    area_ratio = len(dd_area_ndj) / len(dd_area)
    count_sum = np.sum(dd_count)
    money_sum = np.sum(dd_amount)
    return years_mean, area_ratio, count_sum, money_sum

### (2) 이동경로 주변 업종 중 **해당 집단의 선호 업종** 순으로 정렬
- *카드매출_유입고객* 은 고객의 월 소비 데이터를 담고 있기 때문에 특정 집단(연령/성별)의 사람들이 한 달 동안 며칠 몇시 즈음에 어떤 업종에서 얼만큼을 소비했는 지 알 수 있음
- 따라서 앞서 3-3 에서 추가한 이동경로 주변 업종을 지금과 같은 **무작위 상태로 두는 것이 아니라 해당 집단의 사람들이 한 달 동안 소비가 많았던 순**으로 정렬
- 다음 3가지의 경우로 나눠서 분석 진행  

  [1] **"-"**  
      - *전후_방문장소_경도 == '-'* : 유동인구가 없는 경우에는 정렬할 업종이 없음  
      - *연령 == 0* :  10세미만은 *카드매출_유입고객* 에 데이터 존재하지 않기 때문에 제외  
      - *업종소분류_코드 == '주변 상권X'* : 이돋경로 주변에 업종이 존재하지 않기 때문에 정렬할 업종이 없음

</br>

  [2] **"월 방문 업종X"**  
      - 이동경로 주변에 업종은 존재하지만, 즉 *카드매출_가맹점*에는 존재하는 업종이지만 집단, 날짜, 시간대별로 필터링된 *카드매출_유입고객*에는 해당 업종이 없는 경우

</br>


  [3] **"(정렬)"**  
      - 이동경로 주변 업종이 집단, 날짜, 시간대별로 필터링된 *카드매출_유입고객*에도 존재하는 경우, 총 이용건수 > 총 이용금액이 높은 순으로 정렬  
      - 총 이용건수가 같은 경우 총 이용금액이 높은 순으로 정렬

In [44]:
def process_result(indice, result_df, customer_data):

    # *연령 & 성별 & 방문 시간대 & 휴일/평일 & 날짜* 멀티인덱스마다 여러 업종이 붙어있음 (한 cell에 여러 업종이 있기 때문)
    # 따라서 멀티인덱스별로 업종 정렬 진행
    result = result_df.loc[indice, :]

    # 평균_업력, 고객_타지역_비율, 고객_월이용건수, 고객_월이용금액 을 기록할 리스트
    years_mean = []
    area_ratio = []
    count_list = []
    money_list = []

    '''
    1 전후_방문장소_경도 == '-' : 유동인구 X → 3-3에서 추가된 업종이 없음
    2 연령 == 0 : 10세미만은 *카드매출_가맹점* 에 데이터 존재 X
    3 업종소분류 == 주변 상권X → 3-3에서 추가된 업종이 없음
    '''
    if  result.index[0][9] == '-' or result.index[0][13] == 0 or (result['업종소분류_코드'] == '주변 상권X').any():
        result['평균_업력'] = "-"
        result['고객_타지역_비율'] = "-"
        result['고객_월이용건수'] = "-"
        result['고객_월이용금액'] = "-"
   
        result_final = result.reset_index()

        # 정렬할 때만 필요하고 이후에 필요없는 컬럼은 제거
        result_final.drop(['구분_시간대', '연령', '성별_숫자', '구분_휴일평일', '날짜_숫자형'], axis=1, inplace=True)
        return result_final
   
   
    else:

        # *카드매출_유입고객* & *3-3까지 진행한 이동경로 및 소비패턴 데이터* 
        # << 연령 & 성별 & 방문 시간대 & 휴일/평일 & 월 >> 조건 맞춰서 갖고오기
        customer_conditions = (
            (customer_data['기준년월_numeric'] == int(str(result.index[0][16])[4:6])) &   # 날짜 중 월 만 추출해서 비교 (∵ 카드매출_유입고객은 월별 데이터)
            (customer_data['구분_시간대'] == result.index[0][12]) &
            (customer_data['연령'] == result.index[0][13]) &
            (customer_data['성별_numeric'] == result.index[0][14]) &
            (customer_data['구분_휴일평일'] == result.index[0][15])
        )
   
        customer_reduced2 = customer_data[customer_conditions].copy()

        
        # 총 이용건수 > 총 이용금액이 높은 순으로 정렬
        custom_list = customer_reduced2.groupby('업종소분류')[['이용건수', '이용금액']].sum().reset_index()
        custom_list = custom_list.sort_values(by=['이용건수', '이용금액'], ascending=False)

        
        # 1) 3-3에서 추가한 업종 (*카드매출_가맹점* 500m 셀 안에 있는 모든 업종)
        store = result['업종소분류_코드'].tolist()

        # 2) 위에서 정렬한 *카드매출_유입고객 > 업종소분류* 리스트
        custom = custom_list['업종소분류'].tolist()


        
        # 1)에 있는 2)는 정렬된 순서대로 리스트에 저장
        # 즉, 이동경로 주변 업종 중 고객이 방문한 업종을 이용건수가 높은 순서대로 리스트에 저장
        ordered_data = [x for x in custom if x in store]

        # 1)에 없는 2)는 순서 고려하지 않고 리스트에 저장
        # 즉, 이동경로 주변 업종이지만, 해당 집단의 고객이 한 달동안 그 시간에 방문하지 않은 업종을 리스트에 저장
        remaining_data = [x for x in store if x not in custom]

        # 고객이 방문한 업종을 이용건수가 높은 순서대로 저장한 리스트와 고객이 한 달동안 방문하지 않은 업종을 저장한 리스트를 하나로 합치기
        # 즉, 데이터 프레임 상단에는 고객이 방문한 업종을 이용건수가 높은 순서대로, 하단에는 방문하지 않은 업종이 나타나도록 리스트 생성
        final_sorted_data = ordered_data + remaining_data


        
        # 업종별 데이터 처리를 위해 for문 생성
        for i in final_sorted_data:
            
            ''' *카드매출_유입고객*에 존재하는 업종인지 필터링
            - ordered_data에 저장돼있던 업종은 *카드매출_유입고객*에도 존재
            - remaining_data에 저장돼있던 업종은 *카드매출_유입고객*에 존재X → empty 처리 됨
            '''
            dd = customer_reduced2[customer_reduced2['업종소분류'] == i]

            # 이동경로 주변 업종이지만 *카드매출_유입고객*에 존재하지 않는 업종의 경우 다음과 같이 처리
            if dd.empty:
                years_mean.append("월 방문 업종X")
                area_ratio.append("월 방문 업종X")
                count_list.append("월 방문 업종X")
                money_list.append("월 방문 업종X")

            # 이동경로 주변 업종이고 *카드매출_유입고객*에 존재하는 경우 *업력_숫자, 광역시_유입, 이용건수, 이용금액* 을 추출하여
            # (1)에서 정의한 계산함수에 입력
            else:
                # Numpy 배열로 변환하여 Numba 적용
                dd_years = dd['업력_숫자'].values.astype(np.float32)
                dd_area = dd['광역시_유입'].values.astype(np.int8)
                dd_area_ndj = dd[dd['광역시_유입']!=30]['광역시_유입'].values.astype(np.int8)
                dd_count = dd['이용건수'].values.astype(np.int32)
                dd_amount = dd['이용금액'].values.astype(np.float32)

                # 계산 함수 호출
                mean, ratio, count, money = calculate_customer_stats(dd_years, dd_area, dd_area_ndj, dd_count, dd_amount)

                # 결과를 각각의 리스트에 저장
                years_mean.append(f"{mean:.2f}")
                area_ratio.append(f"{ratio:.2f}")
                count_list.append(count)
                money_list.append(money)
   
   
        # 정렬된 순서가 3-3까지 진행한 데이터프레임에도 반영되도록 다음과 같이 처리

        # 앞서 정리한 업종 리스트를 "업종소분류"라는 컬럼을 새로 만들어서 추가
        result['업종소분류'] = final_sorted_data

        # 이후 최종 결과물에서 필요한 멀티인덱스 일부만 리스트로 저장해두기
        result_index = result.reset_index().iloc[:,:12]

        # 기존 데이터프레임에 존재하던 "업종소분류_코드"를 인덱스로 만든 후,
        # final_sorted_data 값을 갖고 있는 "업종소분류"를 기준으로 데이터 정렬
        result = result.set_index('업종소분류_코드').loc[result['업종소분류']].reset_index()
       
        # 앞서 계산했던 리스트들을 기반으로 각각 컬럼 생성
        result['평균_업력'] = years_mean
        result['고객_타지역_비율'] = area_ratio
        result['고객_월이용건수'] = count_list
        result['고객_월이용금액'] = money_list
       
        # 정렬에 사용된 "업종소분류"는 drop (이제 정렬이 완료되었기 때문에 더이상 사용X)
        result.drop('업종소분류', axis=1, inplace=True)

        # 앞서 생성한 멀티인덱스 리스트와 정렬된 데이터를 합치기
        result_final = pd.concat([result_index, result], axis=1)
        return result_final

### (3) 정의한 함수를 날짜별로 병렬 처리

In [47]:
# 날짜별로 병렬 처리
grouped_by_date = result_df.groupby('날짜')

results_parallel = Parallel(n_jobs=-1)(
    delayed(lambda date, daily_data: pd.concat(
        [process_result(indice, daily_data, customer_data) for indice in daily_data.index.unique().tolist()])
    )(date, daily_data) for date, daily_data in grouped_by_date)

# 병렬 처리 후 결과 병합
result_total = pd.concat(results_parallel)

In [49]:
result_total

,날짜,공휴일,주말,연령대,성별,성심당_유동인구,성심당_방문시간,전후_방문시간대,전후_방문장소_위도,전후_방문장소_경도,...,이동거리(km),업종소분류_코드,업종소분류_항목명,업종_일판매건수,업종_일매출,업종_일매출_지역화폐,평균_업력,고객_타지역_비율,고객_월이용건수,고객_월이용금액
0,2023-08-12,공휴일 아님,주말,20대,여성,0.00,8,-,-,-,...,-,-,-,-,-,-,-,-,-,-
0,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,0,36.319641,127.395887,...,-,47122.0,체인화 편의점,116,796820,84070,6.23,0.23,2187,14174170.0
1,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,0,36.319641,127.395887,...,-,56111.0,한식 일반 음식점업,80,2331000,247000,6.10,0.23,1084,44813040.0
2,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,0,36.319641,127.395887,...,-,47121.0,슈퍼마켓,114,1682660,265110,9.45,0.14,185,2173160.0
3,2023-08-12,공휴일 아님,주말,20대,여성,0.80,9,0,36.319641,127.395887,...,-,47129.0,기타 음ㆍ식료품 위주 종합 소매업,19,142350,21800,3.40,0.06,138,567500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,18,36.320633,127.414255,...,1.414298,21300.0,의료용품 및 기타 의약 관련제품 제조업,3,425500,0,3.00,0.00,0,0.0
0,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,19,36.329262,127.428114,...,1.570126,주변 상권X,주변 상권X,주변 상권X,주변 상권X,주변 상권X,-,-,-,-
0,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,20,36.32746,127.428127,...,0.200012,주변 상권X,주변 상권X,주변 상권X,주변 상권X,주변 상권X,-,-,-,-
0,2023-08-12,공휴일 아님,주말,20대,여성,30.92,21,22,36.325183,127.423131,...,0.471728,주변 상권X,주변 상권X,주변 상권X,주변 상권X,주변 상권X,-,-,-,-


# 4 이동경로 및 소비패턴 데이터 저장

In [ ]:
result_total.to_csv(os.path.join(path, '이동경로 및 소비패턴.csv'), encoding = 'cp949', index = False)